In [1]:
import numpy as np
import tensorflow as tf

In [2]:
v1 = np.array([1, 2, 3], dtype=float)
v2 = np.array([1, 2, 3.5], dtype=float)

print("-- Inputs --")
print("v1 :", v1)
print("v2 :", v2, "\n")

def cosine_similarity(v1, v2):
    numerator = tf.math.reduce_sum(v1*v2) # takes the dot product between v1 and v2. Equivalent to np.dot(v1, v2)
    denominator = tf.math.sqrt(tf.math.reduce_sum(v1*v1) * tf.math.reduce_sum(v2*v2))
    return numerator / denominator

print("-- Outputs --")
print("cosine similarity :", cosine_similarity(v1, v2).numpy())

-- Inputs --
v1 : [1. 2. 3.]
v2 : [1.  2.  3.5] 

-- Outputs --
cosine similarity : 0.9974086507360697


In [3]:
# Two batches of vectors example
# Input data

v1_1 = np.array([1.0, 2.0, 3.0])
v1_2 = np.array([9.0, 8.0, 7.0])
v1_3 = np.array([-1.0, -4.0, -2.0])
v1_4 = np.array([1.0, -7.0, 2.0])
v1 = np.vstack([v1_1, v1_2, v1_3, v1_4])

v2_1 = v1_1 + np.random.normal(0, 2, 3)  # add some noise to create approximate duplicate
v2_2 = v1_2 + np.random.normal(0, 2, 3)
v2_3 = v1_3 + np.random.normal(0, 2, 3)
v2_4 = v1_4 + np.random.normal(0, 2, 3)
v2 = np.vstack([v2_1, v2_2, v2_3, v2_4])

print("-- Inputs --")
print(f"v1 :\n{v1}\n")
print(f"v2 :\n{v2}\n")

# Batch sizes must match
b = len(v1)
print(f"Batch sizes match : {b == len(v2)}\n")

# Similarity scores

# Option 1 : nested loops and the cosine similarity function
sim_1 = np.zeros([b, b])  # empty array to take similarity scores
# Loop
for row in range(0, sim_1.shape[0]):
    for col in range(0, sim_1.shape[1]):
        sim_1[row, col] = cosine_similarity(v2[row], v1[col]).numpy()

print("-- Outputs --")
print("Option 1 : loop")
print(sim_1)

-- Inputs --
v1 :
[[ 1.  2.  3.]
 [ 9.  8.  7.]
 [-1. -4. -2.]
 [ 1. -7.  2.]]

v2 :
[[ 3.45083126  2.98917302  1.31933952]
 [ 9.62350386  5.99327392  6.44324358]
 [-0.90127254 -2.53262805 -0.49063025]
 [ 2.16346773 -6.56657597  1.70029122]]

Batch sizes match : True

-- Outputs --
Option 1 : loop
[[ 0.75287816  0.97000839 -0.8286579  -0.42479658]
 [ 0.83906886  0.98916255 -0.77785994 -0.20289944]
 [-0.72750723 -0.83568191  0.95932141  0.7891139 ]
 [-0.22030203 -0.21337035  0.63451281  0.9849096 ]]


In [4]:
# Option 2 : vector normalization and dot product
def norm(x):
    return tf.math.l2_normalize(x, axis=1) # use tensorflow built in normalization

sim_2 = tf.linalg.matmul(norm(v2), norm(v1), transpose_b=True)

print("-- Outputs --")
print("Option 2 : vector normalization and dot product")
print(sim_2, "\n")

# Check
print(f"Outputs are the same : {np.allclose(sim_1, sim_2)}")

-- Outputs --
Option 2 : vector normalization and dot product
tf.Tensor(
[[ 0.75287816  0.97000839 -0.8286579  -0.42479658]
 [ 0.83906886  0.98916255 -0.77785994 -0.20289944]
 [-0.72750723 -0.83568191  0.95932141  0.7891139 ]
 [-0.22030203 -0.21337035  0.63451281  0.9849096 ]], shape=(4, 4), dtype=float64) 

Outputs are the same : True


In [5]:
sim_hardcoded = np.array(
    [
        [0.9, -0.8, 0.3, -0.5],
        [-0.4, 0.5, 0.1, -0.1],
        [0.3, 0.1, -0.4, -0.8],
        [-0.5, -0.2, -0.7, 0.5],
    ]
)

sim = sim_hardcoded

In [6]:
print("-- Inputs --")
print(f"sim:")
print(sim)
print(f"shape: {sim.shape}\n")

-- Inputs --
sim:
[[ 0.9 -0.8  0.3 -0.5]
 [-0.4  0.5  0.1 -0.1]
 [ 0.3  0.1 -0.4 -0.8]
 [-0.5 -0.2 -0.7  0.5]]
shape: (4, 4)



In [7]:
# Positives i.e. all the s(A, P) values

sim_ap = np.diag(sim)
sim_ap

array([ 0.9,  0.5, -0.4,  0.5])

In [8]:
# Negatives i.e. all the s(A, N) values

sim_an = sim - sim_ap
sim_an

array([[ 0. , -1.3,  0.7, -1. ],
       [-1.3,  0. ,  0.5, -0.6],
       [-0.6, -0.4,  0. , -1.3],
       [-1.4, -0.7, -0.3,  0. ]])

In [12]:
# Mean negative

mean_neg = np.sum(sim_an, axis=1, keepdims=True) / (b - 1)
mean_neg

array([[-0.53333333],
       [-0.46666667],
       [-0.76666667],
       [-0.8       ]])

In [15]:
# Closest negative
# Max s(A, N) that is <- s(A, P) for each row

mask1 = np.identity(b) == 1
mask1

array([[ True, False, False, False],
       [False,  True, False, False],
       [False, False,  True, False],
       [False, False, False,  True]])

In [17]:
mask2 = sim_an > sim_ap.reshape(b, 1)
mask2

array([[False, False, False, False],
       [False, False, False, False],
       [False, False,  True, False],
       [False, False, False, False]])

In [18]:
mask = mask1 | mask2
mask

array([[ True, False, False, False],
       [False,  True, False, False],
       [False, False,  True, False],
       [False, False, False,  True]])

In [19]:
sim_an_masked = np.copy(sim_an)
sim_an_masked

array([[ 0. , -1.3,  0.7, -1. ],
       [-1.3,  0. ,  0.5, -0.6],
       [-0.6, -0.4,  0. , -1.3],
       [-1.4, -0.7, -0.3,  0. ]])

In [22]:
sim_an_masked[mask] = -2
sim_an_masked

array([[-2. , -1.3,  0.7, -1. ],
       [-1.3, -2. ,  0.5, -0.6],
       [-0.6, -0.4, -2. , -1.3],
       [-1.4, -0.7, -0.3, -2. ]])

In [23]:
closest_neg = np.max(sim_an_masked, axis = 1, keepdims=True)
closest_neg

array([[ 0.7],
       [ 0.5],
       [-0.4],
       [-0.3]])

In [25]:
# Losses
alpha = 0.25
l_1 = tf.maximum(mean_neg - sim_ap + alpha, 0)
l_1

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.        , 0.        , 0.11666667, 0.        ],
       [0.        , 0.        , 0.18333333, 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])>

In [26]:
l_2 = tf.maximum(closest_neg - sim_ap + alpha, 0)
l_2

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.05, 0.45, 1.35, 0.45],
       [0.  , 0.25, 1.15, 0.25],
       [0.  , 0.  , 0.25, 0.  ],
       [0.  , 0.  , 0.35, 0.  ]])>

In [27]:
l_full = l_1 + l_2

In [28]:
l_full

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.05      , 0.45      , 1.46666667, 0.45      ],
       [0.        , 0.25      , 1.33333333, 0.25      ],
       [0.        , 0.        , 0.25      , 0.        ],
       [0.        , 0.        , 0.35      , 0.        ]])>

In [30]:
cost = tf.math.reduce_sum(l_full)
cost

<tf.Tensor: shape=(), dtype=float64, numpy=4.85>